In [807]:
#全データに対応できる汎用的なモデルを作成する
#深層TSKファジィ推論モデル(0次元の積層構造）について示す
#深層TSKファジィ推論モデル（０次元の積層構造）について、特徴量選択はランダムで行うが、ルール選択をランダムで行うか全て網羅すべきかを考える
#ここではルール選択をランダムで行う場合を考える
#とりあえずは学習回数の上限で学習をストップする考えで行う
import numpy as np
import pandas as pd
import random
import math
import sys
import time

#あらかじめ深層学習の層数を設定しておく
NUMBER_OF_LAYER = 3

In [808]:
#各種データを読み込み、読み込んだデータに対して正規化を行う
#そしてnumpy配列に変換しておく
original_data = pd.read_csv('fuzzy_diabetes2_1.csv')

#まず例外カラム名の制定
#for clm in original_data.columns:
    #if clm not in exception_columns:
        #original_data[clm] = (original_data[clm] - original_data[clm].min()) / (original_data[clm].max() - original_data[clm].min())


np_original_data = np.array(original_data)

In [809]:
#元データの入力データと出力データの合計の属性数を求める
#また入力変数の属性数を求めるが、ここでは出力変数の属性数が1であることを仮定している
NUMBER_OF_INPUT_OUTPUT = len(teacher_data[0,:])
NUMBER_OF_INPUT = NUMBER_OF_INPUT_OUTPUT - 1

#またファジィ分割数を求める
#ここではファジィ分割数・ファジィルール数は各層で同じであると仮定する
NUMBER_OF_FUZZY_PARTITION = 3
NUMBER_OF_FUZZY_RULE = 8

#次に各層における入力属性の候補数を設定する
NUMBER_OF_INPUT_LAYER1 = 5
NUMBER_OF_INPUT_LAYER2 = 6 #ただし、1層目の出力は確定で入力変数として扱う
NUMBER_OF_INPUT_LAYER3 = 7 #ただし、2層目の出力は確定で入力変数として扱う

In [810]:
#教師データ数とテストデータ数を決定する
TEACHER_SIZE_RATIO = 5
TEST_SIZE_RATIO = 5
DATA_SIZE = len(np_original_data)
TEACHER_SIZE = ((int)(len(np_original_data) * ((TEACHER_SIZE_RATIO) / (TEACHER_SIZE_RATIO + TEST_SIZE_RATIO)))) + 1
TEST_SIZE = len(np_original_data) - TEACHER_SIZE

In [811]:
#1層目で用いる教師データを作成する
#テストデータはテスト時に作成する
teacher_input_layer1 = np_original_data[:TEACHER_SIZE, :NUMBER_OF_INPUT]
teacher_output_layer1 = np_original_data[:TEACHER_SIZE, NUMBER_OF_INPUT]

In [812]:
#1層目で用いる特徴量選択行列とルール結合行列を決定する
features_selection_matrix_layer1 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1))
for i in range(NUMBER_OF_FUZZY_RULE):
    while True:
        for j in range(NUMBER_OF_INPUT_LAYER1):
            if random.random() < 0.5:
                features_selection_matrix_layer1[i,j] = 0
            else:
                features_selection_matrix_layer1[i,j] = 1
        if 1 in features_selection_matrix_layer1[i,:]:
            break
#次にルール結合行列を設定する
rule_combination_matrix_layer1 = np.zeros((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1, NUMBER_OF_FUZZY_PARTITION))
rule_partition_random = [1, 2, 3]
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER1):
        if (random.choice(rule_partition_random) == 1):
            rule_combination_matrix_layer1[i,j,0] = 1
        elif(random.choice(rule_partition_random) == 2):
            rule_combination_matrix_layer1[i,j,1] = 1
        else:
            rule_combination_matrix_layer1[i,j,2] = 1
print(features_selection_matrix_layer1)
print(rule_combination_matrix_layer1)

[[0. 0. 0. 1. 1.]
 [0. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]]
[[[0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[0. 0. 1.]
  [0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]]

 [[1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 1. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]]]


In [813]:
#前件部と後件部の初期値を設定する
#まずは一層目に対する前件部と後件部の初期値を設定する
antecedent_center_layer1 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1))
antecedent_broad_layer1 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1))
consequent_layer1 = np.empty(NUMBER_OF_FUZZY_RULE)

#まずは後件部実数値の初期設定を行う
#このプログラムでは後件部は0次式で表している
for i in range(NUMBER_OF_FUZZY_RULE):
    consequent_layer1[i] = 0.5
#前件部を初期設定する
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER1):
        if rule_combination_matrix_layer1[i,j,0] == 1:
            antecedent_center_layer1[i,j] = 0
            antecedent_broad_layer1[i,j] = 1
        elif rule_combination_matrix_layer1[i,j,1] == 1:
            antecedent_center_layer1[i,j] = 0.5
            antecedent_broad_layer1[i,j] = 0.5
        else:
            antecedent_center_layer1[i,j] = 1
            antecedent_broad_layer1[i,j] = 1
antecedent_center_layer1

array([[0.5, 0. , 0. , 1. , 0. ],
       [1. , 0.5, 0. , 1. , 0.5],
       [0. , 1. , 0. , 1. , 0. ],
       [0. , 0.5, 1. , 0. , 1. ],
       [1. , 0.5, 1. , 0.5, 0.5],
       [1. , 1. , 1. , 1. , 1. ],
       [0.5, 0. , 1. , 1. , 1. ],
       [0. , 0. , 0. , 0. , 1. ]])

In [814]:
%%time
#1層目の学習を行う
#前件部の学習は行わず、後件部の学習のみを行う

#学習回数、学習係数を設定する
TRAIN_TIME = 1000
LEARNING_CONSEQUENT = 0.0001

#平均二条誤差を設定する
mean_squared_error_layer1 = 0

#適合度・メンバシップ関数値を初期化する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1))

for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER1):
            if(teacher_input_layer1[i,k] >= antecedent_center_layer1[j,k] - antecedent_broad_layer1[j,k]) and (teacher_input_layer1[i,k] <= antecedent_center_layer1[j,k]) and (features_selection_matrix_layer1[j,k] == 1):
                membership_function[i,j,k] = (teacher_input_layer1[i,k] - (antecedent_center_layer1[j,k] - antecedent_broad_layer1[j,k])) / antecedent_broad_layer1[j,k]
            elif(teacher_input_layer1[i,k] > antecedent_center_layer1[j,k]) and (teacher_input_layer1[i,k] <= antecedent_center_layer1[j,k] + antecedent_broad_layer1[j,k]) and (features_selection_matrix_layer1[j,k] == 1):
                membership_function[i,j,k] = -(teacher_input_layer1[i,k] - (antecedent_center_layer1[j,k] + antecedent_broad_layer1[j,k])) / antecedent_broad_layer1[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER1):
            adaptability[i,j] = adaptability[i,j] * membership_function[i,j,k]
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer1 = np.empty(TEACHER_SIZE)
    for i in range(TEACHER_SIZE):
        #各データの予測結果を求める
        output_layer1[i] = np.sum(np.dot(adaptability[i], consequent_layer1)) / np.sum(adaptability[i])
        if(time == TRAIN_TIME - 1):
            mean_squared_error_layer1 = mean_squared_error_layer1 + (output_layer1[i] - teacher_output_layer1[i]) ** 2
            if(i == TEACHER_SIZE - 1):
                mean_squared_error_layer1 = mean_squared_error_layer1 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer1)
        #出力を求めた後、後件部を更新する
        for j in range(NUMBER_OF_FUZZY_RULE):
            consequent_layer1[j] = consequent_layer1[j] + (LEARNING_CONSEQUENT * adaptability[i,j] / np.sum(adaptability[i])) * (teacher_output_layer1[i] - output_layer1[i])

平均二条誤差
0.11603788285373778
Wall time: 3.05 s


In [815]:
#2層目で用いる教師データを作成する
#具体的には1層目で用いた教師データに1層目の出力を追加する
teacher_input_layer2 = np.insert(teacher_input_layer1, NUMBER_OF_INPUT, output_layer1, axis=1)
teacher_output_layer2 = np.array(teacher_output_layer1)

In [816]:
#2層目で用いる特徴量選択行列とルール結合行列を作成する
features_selection_matrix_layer2 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2))
for i in range(NUMBER_OF_FUZZY_RULE):
    while True:
        for j in range(NUMBER_OF_INPUT_LAYER2-1):
            if random.random() < 0.5:
                features_selection_matrix_layer2[i,j] = 0
            else:
                features_selection_matrix_layer2[i,j] = 1
        if 1 in features_selection_matrix_layer2[i,:]:
            break
#さらにここで2層目の特徴量には必ず1層目の出力を用いる
features_selection_matrix_layer2[:, NUMBER_OF_INPUT_LAYER2-1] = 1
#次にルール結合行列を設定する
rule_combination_matrix_layer2 = np.zeros((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2, NUMBER_OF_FUZZY_PARTITION))
rule_partition_random = [1, 2, 3]
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER2):
        if (random.choice(rule_partition_random) == 1):
            rule_combination_matrix_layer2[i,j,0] = 1
        elif(random.choice(rule_partition_random) == 2):
            rule_combination_matrix_layer2[i,j,1] = 1
        else:
            rule_combination_matrix_layer2[i,j,2] = 1
print(features_selection_matrix_layer2)
print(rule_combination_matrix_layer2)

[[1. 1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 1.]
 [0. 1. 1. 1. 1. 1.]
 [1. 0. 0. 1. 0. 1.]]
[[[0. 0. 1.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]]

 [[1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[0. 1. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [1. 0. 0.]]]


In [817]:
#2層目で用いる前件部と後件部の値の初期設定を行う
antecedent_center_layer2 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2))
antecedent_broad_layer2 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2))
consequent_layer2 = np.empty(NUMBER_OF_FUZZY_RULE)

#まずは後件部実数値の初期設定を行う
#このプログラムでは後件部は0次式で表している
for i in range(NUMBER_OF_FUZZY_RULE):
    consequent_layer2[i] = 0.5
#前件部を初期設定する
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER2):
        if rule_combination_matrix_layer2[i,j,0] == 1:
            antecedent_center_layer2[i,j] = 0
            antecedent_broad_layer2[i,j] = 1
        elif rule_combination_matrix_layer2[i,j,1] == 1:
            antecedent_center_layer2[i,j] = 0.5
            antecedent_broad_layer2[i,j] = 0.5
        else:
            antecedent_center_layer2[i,j] = 1
            antecedent_broad_layer2[i,j] = 1
antecedent_center_layer2

array([[1. , 1. , 0.5, 0.5, 1. , 1. ],
       [1. , 0. , 0. , 0. , 0.5, 1. ],
       [0. , 0. , 0. , 0. , 1. , 0.5],
       [0. , 1. , 0.5, 0.5, 0.5, 1. ],
       [0.5, 1. , 0.5, 1. , 0. , 1. ],
       [0. , 0.5, 1. , 0. , 1. , 0. ],
       [0. , 0. , 1. , 1. , 1. , 1. ],
       [0.5, 0. , 0. , 1. , 0. , 0. ]])

In [818]:
%%time
#2層目の学習を行う
#学習回数、学習係数を設定する
#平均二条誤差を設定する
mean_squared_error_layer2 = 0

#適合度・メンバシップ関数値を初期化する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2))

for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER2):
            if(teacher_input_layer2[i,k] >= antecedent_center_layer2[j,k] - antecedent_broad_layer2[j,k]) and (teacher_input_layer2[i,k] <= antecedent_center_layer2[j,k]) and (features_selection_matrix_layer2[j,k] == 1):
                membership_function[i,j,k] = (teacher_input_layer2[i,k] - (antecedent_center_layer2[j,k] - antecedent_broad_layer2[j,k])) / antecedent_broad_layer2[j,k]
            elif(teacher_input_layer2[i,k] > antecedent_center_layer2[j,k]) and (teacher_input_layer2[i,k] <= antecedent_center_layer2[j,k] + antecedent_broad_layer2[j,k]) and (features_selection_matrix_layer2[j,k] == 1):
                membership_function[i,j,k] = -(teacher_input_layer2[i,k] - (antecedent_center_layer2[j,k] + antecedent_broad_layer2[j,k])) / antecedent_broad_layer2[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER2):
            adaptability[i,j] = adaptability[i,j] * membership_function[i,j,k]
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer2 = np.empty(TEACHER_SIZE)
    for i in range(TEACHER_SIZE):
        #各データの予測結果を求める
        output_layer2[i] = np.sum(np.dot(adaptability[i], consequent_layer2)) / np.sum(adaptability[i])
        if(time == TRAIN_TIME - 1):
            mean_squared_error_layer2 = mean_squared_error_layer2 + (output_layer2[i] - teacher_output_layer2[i]) ** 2
            if(i == TEACHER_SIZE - 1):
                mean_squared_error_layer2 = mean_squared_error_layer2 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer2)
        #出力を求めた後、後件部を更新する
        for j in range(NUMBER_OF_FUZZY_RULE):
            consequent_layer2[j] = consequent_layer2[j] + (LEARNING_CONSEQUENT * adaptability[i,j] / np.sum(adaptability[i])) * (teacher_output_layer2[i] - output_layer2[i])

平均二条誤差
0.1582461479956351
Wall time: 3.05 s


In [819]:
#3層目で用いる教師データを設定する
#具体的には2層目の出力を3層目の入力として用いる
teacher_input_layer3 = np.insert(teacher_input_layer2, NUMBER_OF_INPUT+1, output_layer2, axis=1)
teacher_output_layer3 = np.array(teacher_output_layer2)

In [820]:
#3層目で用いる特徴量選択行列とルール結合行列を作成する
features_selection_matrix_layer3 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3))
for i in range(NUMBER_OF_FUZZY_RULE):
    while True:
        for j in range(NUMBER_OF_INPUT_LAYER3-1):
            if random.random() < 0.5:
                features_selection_matrix_layer3[i,j] = 0
            else:
                features_selection_matrix_layer3[i,j] = 1
        if 1 in features_selection_matrix_layer3[i,:]:
            break
#さらにここで2層目の特徴量には必ず1層目の出力を用いる
features_selection_matrix_layer3[:, NUMBER_OF_INPUT_LAYER3-1] = 1
#次にルール結合行列を設定する
rule_combination_matrix_layer3 = np.zeros((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3, NUMBER_OF_FUZZY_PARTITION))
rule_partition_random = [1, 2, 3]
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER3):
        if (random.choice(rule_partition_random) == 1):
            rule_combination_matrix_layer3[i,j,0] = 1
        elif(random.choice(rule_partition_random) == 2):
            rule_combination_matrix_layer3[i,j,1] = 1
        else:
            rule_combination_matrix_layer3[i,j,2] = 1
print(features_selection_matrix_layer3)
print(rule_combination_matrix_layer3)

[[0. 1. 0. 0. 1. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 0. 0. 1.]
 [1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 1. 1. 0. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1.]]
[[[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]]

 [[1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[0. 0. 1.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]]

 [[0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]]]


In [821]:
#3層目で用いる前件部と後件部の値を設定する
antecedent_center_layer3 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3))
antecedent_broad_layer3 = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3))
consequent_layer3 = np.empty(NUMBER_OF_FUZZY_RULE)

#まずは後件部実数値の初期設定を行う
#このプログラムでは後件部は0次式で表している
for i in range(NUMBER_OF_FUZZY_RULE):
    consequent_layer3[i] = 0.5
#前件部を初期設定する
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT_LAYER3):
        if rule_combination_matrix_layer3[i,j,0] == 1:
            antecedent_center_layer3[i,j] = 0
            antecedent_broad_layer3[i,j] = 1
        elif rule_combination_matrix_layer3[i,j,1] == 1:
            antecedent_center_layer3[i,j] = 0.5
            antecedent_broad_layer3[i,j] = 0.5
        else:
            antecedent_center_layer3[i,j] = 1
            antecedent_broad_layer3[i,j] = 1
antecedent_center_layer3

array([[0. , 0. , 0. , 1. , 1. , 0. , 1. ],
       [1. , 0. , 1. , 1. , 0. , 1. , 0.5],
       [0. , 1. , 1. , 0. , 1. , 1. , 0. ],
       [1. , 0.5, 1. , 0. , 1. , 0. , 1. ],
       [1. , 0. , 0.5, 1. , 0. , 0.5, 1. ],
       [0.5, 1. , 0. , 1. , 0. , 1. , 0.5],
       [1. , 0. , 1. , 0.5, 0. , 1. , 0. ],
       [0. , 0.5, 1. , 0.5, 1. , 1. , 0. ]])

In [822]:
%%time
#3層目の学習を行う
#平均二条誤差を設定する
mean_squared_error_layer3 = 0

#適合度・メンバシップ関数値を初期化する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3))

for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER3):
            if(teacher_input_layer3[i,k] >= antecedent_center_layer3[j,k] - antecedent_broad_layer3[j,k]) and (teacher_input_layer3[i,k] <= antecedent_center_layer3[j,k]) and (features_selection_matrix_layer3[j,k] == 1):
                membership_function[i,j,k] = (teacher_input_layer3[i,k] - (antecedent_center_layer3[j,k] - antecedent_broad_layer3[j,k])) / antecedent_broad_layer3[j,k]
            elif(teacher_input_layer3[i,k] > antecedent_center_layer3[j,k]) and (teacher_input_layer3[i,k] <= antecedent_center_layer3[j,k] + antecedent_broad_layer3[j,k]) and (features_selection_matrix_layer3[j,k] == 1):
                membership_function[i,j,k] = -(teacher_input_layer3[i,k] - (antecedent_center_layer3[j,k] + antecedent_broad_layer3[j,k])) / antecedent_broad_layer3[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER3):
            adaptability[i,j] = adaptability[i,j] * membership_function[i,j,k]
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer3 = np.empty(TEACHER_SIZE)
    for i in range(TEACHER_SIZE):
        #各データの予測結果を求める
        output_layer3[i] = np.sum(np.dot(adaptability[i], consequent_layer3)) / np.sum(adaptability[i])
        if(time == TRAIN_TIME - 1):
            mean_squared_error_layer3 = mean_squared_error_layer3 + (output_layer3[i] - teacher_output_layer3[i]) ** 2
            if(i == TEACHER_SIZE - 1):
                mean_squared_error_layer3 = mean_squared_error_layer3 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer3)
        #出力を求めた後、後件部を更新する
        for j in range(NUMBER_OF_FUZZY_RULE):
            consequent_layer3[j] = consequent_layer3[j] + (LEARNING_CONSEQUENT * adaptability[i,j] / np.sum(adaptability[i])) * (teacher_output_layer3[i] - output_layer3[i])

平均二条誤差
0.1319718991614914
Wall time: 2.99 s


In [823]:
%%time
#1層目のテストを行う
#まず1層目におけるテストデータの作成を行う
test_input_layer1 = np_original_data[TEACHER_SIZE:, :NUMBER_OF_INPUT]
test_output_layer1 = np_original_data[TEACHER_SIZE:, NUMBER_OF_INPUT]

#平均二条誤差を設定する
test_mean_squared_error_layer1 = 0

#適合度・メンバシップ関数値を初期化する
test_adaptability = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE))
test_membership_function = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER1))

for i in range(TEST_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER1):
            if(test_input_layer1[i,k] >= antecedent_center_layer1[j,k] - antecedent_broad_layer1[j,k]) and (test_input_layer1[i,k] <= antecedent_center_layer1[j,k]) and (features_selection_matrix_layer1[j,k] == 1):
                test_membership_function[i,j,k] = (test_input_layer1[i,k] - (antecedent_center_layer1[j,k] - antecedent_broad_layer1[j,k])) / antecedent_broad_layer1[j,k]
            elif(test_input_layer1[i,k] > antecedent_center_layer1[j,k]) and (test_input_layer1[i,k] <= antecedent_center_layer1[j,k] + antecedent_broad_layer1[j,k]) and (features_selection_matrix_layer1[j,k] == 1):
                test_membership_function[i,j,k] = -(test_input_layer1[i,k] - (antecedent_center_layer1[j,k] + antecedent_broad_layer1[j,k])) / antecedent_broad_layer1[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER1):
            test_adaptability[i,j] = test_adaptability[i,j] * test_membership_function[i,j,k]
#各データの予測結果を格納する
output_layer1 = np.empty(TEST_SIZE)
for i in range(TEST_SIZE):
    #各データの予測結果を求める
    output_layer1[i] = np.sum(np.dot(test_adaptability[i], consequent_layer1)) / np.sum(test_adaptability[i])
    test_mean_squared_error_layer1 = test_mean_squared_error_layer1 + (output_layer1[i] - test_output_layer1[i]) ** 2
    if(i == TEST_SIZE - 1):
        test_mean_squared_error_layer1 = test_mean_squared_error_layer1 / TEST_SIZE
        print('平均二条誤差')
        print(test_mean_squared_error_layer1)

平均二条誤差
0.12759237880504248
Wall time: 6.98 ms


In [824]:
%%time
#2層目のテストを行う
#まず2層目におけるテストデータを作成する
test_input_layer2 = np.insert(test_input_layer1, NUMBER_OF_INPUT, output_layer1, axis=1)
test_output_layer2 = np.array(test_output_layer1)

#平均二条誤差を設定する
test_mean_squared_error_layer2 = 0

#適合度・メンバシップ関数値を初期化する
test_adaptability = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE))
test_membership_function = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER2))

for i in range(TEST_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER2):
            if(test_input_layer2[i,k] >= antecedent_center_layer2[j,k] - antecedent_broad_layer2[j,k]) and (test_input_layer2[i,k] <= antecedent_center_layer2[j,k]) and (features_selection_matrix_layer2[j,k] == 1):
                test_membership_function[i,j,k] = (test_input_layer2[i,k] - (antecedent_center_layer2[j,k] - antecedent_broad_layer2[j,k])) / antecedent_broad_layer2[j,k]
            elif(test_input_layer2[i,k] > antecedent_center_layer2[j,k]) and (test_input_layer2[i,k] <= antecedent_center_layer2[j,k] + antecedent_broad_layer2[j,k]) and (features_selection_matrix_layer2[j,k] == 1):
                test_membership_function[i,j,k] = -(test_input_layer2[i,k] - (antecedent_center_layer2[j,k] + antecedent_broad_layer2[j,k])) / antecedent_broad_layer2[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER2):
            test_adaptability[i,j] = test_adaptability[i,j] * test_membership_function[i,j,k]
#各データの予測結果を格納する
output_layer2 = np.empty(TEST_SIZE)
for i in range(TEST_SIZE):
    #各データの予測結果を求める
    output_layer2[i] = np.sum(np.dot(test_adaptability[i], consequent_layer2)) / np.sum(test_adaptability[i])
    test_mean_squared_error_layer2 = test_mean_squared_error_layer2 + (output_layer2[i] - test_output_layer2[i]) ** 2
    if(i == TEST_SIZE - 1):
        test_mean_squared_error_layer2 = test_mean_squared_error_layer2 / TEST_SIZE
        print('平均二条誤差')
        print(test_mean_squared_error_layer2)

平均二条誤差
0.18043948211469768
Wall time: 12 ms


In [825]:
%%time
#3層目のテストを行う
#まず3層目におけるテストデータを作成する
test_input_layer3 = np.insert(test_input_layer2, NUMBER_OF_INPUT, output_layer2, axis=1)
test_output_layer3 = np.array(test_output_layer2)

#平均二条誤差を設定する
test_mean_squared_error_layer3 = 0

#適合度・メンバシップ関数値を初期化する
test_adaptability = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE))
test_membership_function = np.ones((TEST_SIZE, NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT_LAYER3))

for i in range(TEST_SIZE):
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT_LAYER3):
            if(test_input_layer3[i,k] >= antecedent_center_layer3[j,k] - antecedent_broad_layer3[j,k]) and (test_input_layer3[i,k] <= antecedent_center_layer3[j,k]) and (features_selection_matrix_layer3[j,k] == 1):
                test_membership_function[i,j,k] = (test_input_layer3[i,k] - (antecedent_center_layer3[j,k] - antecedent_broad_layer3[j,k])) / antecedent_broad_layer3[j,k]
            elif(test_input_layer3[i,k] > antecedent_center_layer3[j,k]) and (test_input_layer3[i,k] <= antecedent_center_layer3[j,k] + antecedent_broad_layer3[j,k]) and (features_selection_matrix_layer3[j,k] == 1):
                test_membership_function[i,j,k] = -(test_input_layer3[i,k] - (antecedent_center_layer3[j,k] + antecedent_broad_layer3[j,k])) / antecedent_broad_layer3[j,k]
        for k in range(NUMBER_OF_INPUT_LAYER3):
            test_adaptability[i,j] = test_adaptability[i,j] * test_membership_function[i,j,k]
#各データの予測結果を格納する
output_layer3 = np.empty(TEST_SIZE)
for i in range(TEST_SIZE):
    #各データの予測結果を求める
    output_layer3[i] = np.sum(np.dot(test_adaptability[i], consequent_layer3)) / np.sum(test_adaptability[i])
    test_mean_squared_error_layer3 = test_mean_squared_error_layer3 + (output_layer3[i] - test_output_layer3[i]) ** 2
    if(i == TEST_SIZE - 1):
        test_mean_squared_error_layer3 = test_mean_squared_error_layer3 / TEST_SIZE
        print('平均二条誤差')
        print(test_mean_squared_error_layer3)

平均二条誤差
0.17617541182486224
Wall time: 13 ms
